In [3]:
%pip install google-cloud-speech
%pip install PyAudio

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for PyAudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.9-universal2-cpython-39/pyaudio
      copying src/pyaudio/__init__.py -> build/lib.macosx-10.9-universal2-cpython-39/pyaudio
      running build_ext
      building 'pyaudio._portaudio' extension
      creating build/temp.macosx-10.9-universal2-cpython-39/src/pyaudio
      clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -iwithsysroot/System/Library/Frameworks/System.framework/PrivateHeaders -iwithsysroot/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python

In [2]:
import pyaudio

# Audio configuration
FORMAT = pyaudio.paInt16  # 16-bit audio format
CHANNELS = 2              # Use 2 channels (Built-in Microphone supports 2)
RATE = 44100              # Common sampling rate
CHUNK = 1024              # Frames per buffer

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open input and output streams
try:
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    output=True,
                    input_device_index=1,  # Built-in Microphone
                    output_device_index=2,  # Built-in Output
                    frames_per_buffer=CHUNK)

    print("Streaming audio... Press Ctrl+C to stop.")
    while True:
        data = stream.read(CHUNK)   # Read from input device
        stream.write(data)          # Write to output device
except Exception as e:
    print(f"Error: {e}")
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


ModuleNotFoundError: No module named 'pyaudio'

In [4]:
import pyaudio

p = pyaudio.PyAudio()

print("Available audio devices:")
for i in range(p.get_device_count()):
    device_info = p.get_device_info_by_index(i)
    print(f"Device {i}: {device_info['name']}")
    print(f"  Max Input Channels: {device_info['maxInputChannels']}")
    print(f"  Max Output Channels: {device_info['maxOutputChannels']}")

p.terminate()


Available audio devices:
Device 0: Rohan’s Iphone Microphone
  Max Input Channels: 1
  Max Output Channels: 0
Device 1: Built-in Microphone
  Max Input Channels: 2
  Max Output Channels: 0
Device 2: Built-in Output
  Max Input Channels: 0
  Max Output Channels: 2


In [5]:
import os
import pyaudio
from google.cloud import speech
from six.moves import queue

# Set your Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"  # Replace with your JSON key file path

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream:
    """Opens a recording stream as a generator yielding audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer,
        )
        self.closed = False
        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            if chunk is None:
                return
            yield chunk

def transcribe_stream():
    """Streams audio to Google Cloud Speech-to-Text and prints the transcriptions."""
    client = speech.SpeechClient()

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        print("Listening for audio... Press Ctrl+C to stop.")
        try:
            for response in responses:
                if not response.results:
                    continue

                result = response.results[0]
                if result.is_final:
                    print(f"Final: {result.alternatives[0].transcript}")
                else:
                    print(f"Interim: {result.alternatives[0].transcript}")
        except KeyboardInterrupt:
            print("Stopped by user.")

if __name__ == "__main__":
    transcribe_stream()


||PaMacCore (AUHAL)|| Error on line 1277: err='-66748', msg=Unknown Error


OSError: [Errno -9986] Internal PortAudio error